Import necessary packages

In [1]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END
from langchain_community.document_loaders import DirectoryLoader, UnstructuredExcelLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_core.runnables import RunnablePassthrough
import nest_asyncio
from langchain import hub
nest_asyncio.apply()


#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "Context_files"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)

#invoke and run the model with the given prompt



#create inputs to the model, telling it what needs to be done. 
#provides system message
system1 = context_gen("system1.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

system3 = context_gen("system3.txt")

system2 = context_gen("system2.txt")

rag_prompt = context_gen("rag_prompt.txt")

class SubQuery(BaseModel):
    '''Search for the geometric definition of a feature across the vector database'''

    sub_query: str = Field(description="The text to be used as a sub-query in the prompt.")   
#First input to the model, breaks query down and provides geometry definition

class YamlText(BaseModel):
    text: str = Field(SQLQuery = "Yaml Text printed one line at a time")

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0, )

In [2]:
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [11]:
'''change the human variable to test different inputs'''
human = ("select all boltholes with a radius of 5mm in the xy plane")

query_analyzer = ChatPromptTemplate.from_messages([("system", rag_prompt), ("human", human)])

chain = query_analyzer | chat 
rag_output = chain.invoke({}, {"tags": ["loop 001"]})
print(rag_output.content)

Yes.

In the query, the term "boltholes" refers to holes that are typically used for bolts. Although the term "boltholes" is not explicitly mentioned in the database schema, it can be inferred that "boltholes" are likely represented as edges with a circular shape and a specific radius.

The query specifies that the boltholes should have a radius of 5mm and be in the xy plane, which can be translated to edges with a radius of 5mm and lying in the xy plane. The database schema contains information about edges, including their radius and position in the global coordinate system, which can be used to identify edges that meet the criteria specified in the query.


In [12]:
#This model sends the user query to the LLM, which then breaks it down into smaller parts if there are words it does not understand"""



prompt = ChatPromptTemplate.from_messages([("system", system2), ("human", human),])
chat_with_tools = chat.bind_tools([SubQuery])
parser = PydanticToolsParser(tools=[SubQuery])
query_analyzer = prompt | chat_with_tools | parser 
output = query_analyzer.invoke({}, {"tags": ["loop 002"]})


strings = str([item for item in output])
print(strings)



[SubQuery(sub_query='what is the definition of a bolthole?'), SubQuery(sub_query='what is the definition of the xy plane?')]


In [ ]:
#chat prompt template for defining the words 
definer = ChatPromptTemplate.from_messages([("system", system3), ("human", strings),])
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, streaming=True)

#using string output parser as the output is a string
output_parser = StrOutputParser()
chain2 = definer | chat 
definition = chain2.invoke({}, {"tags":["loop 003"]})
print(definition.content)


In [ ]:
#Base prompt chain, generates the first yaml version with minimal context
base_prompt = ChatPromptTemplate.from_messages([("system", system1), MessagesPlaceholder(variable_name="messages")])
base_chain = base_prompt | chat 

#This chain ise used for the reflection prompt, which asks the model to critique and improve on the generated YAML text.
reflection_prompt = ChatPromptTemplate.from_messages([("system", refsystem), few_shot_prompt, ("ai", definition.content), MessagesPlaceholder(variable_name="messages"), ])
reflect = reflection_prompt | chat 


In [ ]:
async def generation_node(state: Sequence[BaseMessage]):
    return await base_chain.ainvoke({"messages": state}, {"tags": ["loop 004"]})


async def reflection_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state[0]] + [cls_map[msg.type](content=msg.content) for msg in state[1:]]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

def should_continue(state: List[BaseMessage]):
    if len(state) > 5:
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

async for event in graph.astream([HumanMessage(content=human)]):
    for key, value in event.items():
        if isinstance(value, dict):
            print(value.get('content'))
            print('---')
            print()
        elif isinstance(value, list):
             for item in value:
                if isinstance(item, dict):
                    print(item.get('content'))
                    print('---')
                    print()
                else:
                    print(item.content)
                    print('---')
                    print()
                    


In [10]:
loader = DirectoryLoader("./Context_files")
raw_docs = loader.load()
textsplitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = textsplitter.split_documents(raw_docs)
db = FAISS.from_documents(docs, OpenAIEmbeddings())

retriever = db.as_retriever()

prompt_for_rag = hub.pull("rlm/rag-prompt")

rag_chain = ({"context": retriever, "question": RunnablePassthrough()} | prompt_for_rag | chat | StrOutputParser())

rag_chain.invoke("What is the best way to select a flange?")

'To select a flange, look for planar surfaces with a relatively large area compared to neighboring surfaces in the faces table. Flanges can be identified by their large area and planar type. Consider using commands like "select all faces with radius less than 10mm" to help in the selection process.'